### Importação de configurações e funções

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *

apply_storage_config(spark)

In [0]:
from pyspark.sql.functions import (
    col,
    when,
    lit,
    trim,
    upper,
    lower,
    coalesce,
    concat,
    substring,
    length,
    cast,
    round,
    sum as spark_sum,
    count,
    avg,
    max as spark_max,
    min as spark_min,
    countDistinct,
    current_timestamp,
    year,
    month,
    datediff,
    to_date,
    date_format,
    rank,
    dense_rank,
    lag,
    lead,
    sha2,
    concat_ws,
    ceil
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType
)

### Definição dos paths

In [0]:
path_storage_silver = f"{silver_path}"
path_storage_gold = f"{gold_path}"

print(f"Silver path: {path_storage_silver}")
print(f"Gold path: {path_storage_gold}")

### Criação da tabela de controle de transformações Gold

In [0]:
def criar_tabela_controle_gold():
    """
    Cria tabela para rastrear transformações realizadas na Gold
    """
    spark.sql("""
        CREATE TABLE IF NOT EXISTS gold.transformacoes_processadas (
            tabela_gold STRING,
            data_ultima_transformacao TIMESTAMP,
            total_registros BIGINT,
            data_processamento TIMESTAMP
        )
        USING DELTA
    """)
    print("Tabela de controle de transformações Gold criada!")

criar_tabela_controle_gold()

### Funções auxiliares para transformações

In [0]:
def obter_data_ultima_transformacao_gold(tabela_gold):
    """
    Obtém a data da última transformação realizada para uma tabela na Gold
    """
    try:
        resultado = spark.sql(f"""
            SELECT MAX(data_ultima_transformacao) as ultima_data
            FROM gold.transformacoes_processadas
            WHERE tabela_gold = '{tabela_gold}'
        """).collect()
        
        if resultado and resultado[0].ultima_data:
            return resultado[0].ultima_data
        else:
            return None
    except:
        return None

In [0]:
def registrar_transformacao_gold(tabela_gold, total_registros):
    """
    Registra transformação realizada na Gold
    """
    spark.sql(f"""
        INSERT INTO gold.transformacoes_processadas
        VALUES (
            '{tabela_gold}',
            current_timestamp(),
            {total_registros},
            current_timestamp()
        )
    """)

### Dimensão TEMPO

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO TEMPO")
print("=" * 60)

# Ler dados de EXP e IMP da Silver
df_exp_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/exp_tratado")
df_imp_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/imp_tratado")

# Extrair datas únicas
datas_exp = df_exp_tratado.select("CO_ANO", "CO_MES").distinct()
datas_imp = df_imp_tratado.select("CO_ANO", "CO_MES").distinct()

df_datas_base = datas_exp.union(datas_imp).distinct()

print(f"Total de períodos únicos: {df_datas_base.count()}")

In [0]:
# Criar dimensão tempo com cálculos de trimestre e semestre
dim_tempo = (
    df_datas_base
    .withColumn("CO_TRIMESTRE", ceil(col("CO_MES") / 3))
    .withColumn("CO_SEMESTRE", ceil(col("CO_MES") / 6))
    .withColumn("NO_TRIMESTRE", concat(col("CO_TRIMESTRE"), lit("º Trimestre")))
    .withColumn("NO_SEMESTRE", concat(col("CO_SEMESTRE"), lit("º Semestre")))
    .withColumn("sk_tempo", sha2(concat_ws("||", col("CO_ANO"), col("CO_MES")), 256))
    .withColumn("data_criacao", current_timestamp())
    .orderBy("CO_ANO", "CO_MES")
)

print(f"Dimensão TEMPO criada: {dim_tempo.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    dim_tempo
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_gold}dim_tempo/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_tempo
    USING DELTA
    LOCATION '{path_storage_gold}dim_tempo/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("dim_tempo", dim_tempo.count())

print(" Dimensão TEMPO salva em Gold")

### Dimensão PRODUTO

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO PRODUTO")
print("=" * 60)

# Carregar tabelas de referência da Silver
ncm_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/ncm_tratado")
ncm_sh_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/ncm_sh_tratado")
ncm_isic_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/ncm_isic_tratado")
ncm_cgce_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/ncm_cgce_tratado")
ncm_fat_agreg_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/ncm_fat_agreg_tratado")

print(f" NCM: {ncm_tratado.count()} registros")
print(f" NCM_SH: {ncm_sh_tratado.count()} registros")
print(f" NCM_ISIC: {ncm_isic_tratado.count()} registros")
print(f" NCM_CGCE: {ncm_cgce_tratado.count()} registros")
print(f" NCM_FAT_AGREG: {ncm_fat_agreg_tratado.count()} registros")

In [0]:
# Realizar joins para criar dimensão produto
dim_produto = (
    ncm_tratado
    .join(ncm_sh_tratado, on="CO_SH6", how="left")
    .join(ncm_isic_tratado, on="CO_ISIC_CLASSE", how="left")
    .join(ncm_cgce_tratado, on="CO_CGCE_N3", how="left")
    .join(ncm_fat_agreg_tratado, on="CO_FAT_AGREG", how="left")
    .withColumn("sk_produto", sha2(concat_ws("||", col("CO_NCM")), 256))
    .withColumn("data_criacao", current_timestamp())
)

print(f" Dimensão PRODUTO criada: {dim_produto.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    dim_produto
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_gold}dim_produto/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_produto
    USING DELTA
    LOCATION '{path_storage_gold}dim_produto/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("dim_produto", dim_produto.count())

print("Dimensão PRODUTO salva em Gold")

### Dimensão LOCALIDADE_BR

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO LOCALIDADE_BR")
print("=" * 60)

# Carregar tabelas de referência da Silver
uf_mun_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/uf_mun_tratado")
uf_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/uf_tratado")

print(f"UF_MUN: {uf_mun_tratado.count()} registros")
print(f"UF: {uf_tratado.count()} registros")

In [0]:
%sql
SELECT * FROM silver.uf_tratado

In [0]:
# Realizar join para criar dimensão localidade
dim_localidade_br = (
    uf_mun_tratado
    .join(uf_tratado, on="SG_UF", how="left")
    .select(
        uf_mun_tratado.CO_MUN_GEO,
        uf_mun_tratado.NO_MUN,
        uf_mun_tratado.SG_UF,
        uf_tratado.CO_UF,
        uf_tratado.NO_UF,
        uf_tratado.NO_REGIAO
    )
    .withColumn("sk_localidade", sha2(concat_ws("||", col("NO_UF"), col("CO_MUN_GEO")), 256))
    .withColumn("data_criacao", current_timestamp())
)

print(f" Dimensão LOCALIDADE_BR criada: {dim_localidade_br.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    dim_localidade_br
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_gold}dim_localidade_br/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_localidade_br
    USING DELTA
    LOCATION '{path_storage_gold}dim_localidade_br/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("dim_localidade_br", dim_localidade_br.count())

print(" Dimensão LOCALIDADE_BR salva em Gold")

### Dimensão GEOGRAFIA

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO GEOGRAFIA")
print("=" * 60)

# Carregar tabelas de referência da Silver
pais_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/pais_tratado")
pais_bloco_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/pais_bloco_tratado")

print(f" PAIS: {pais_tratado.count()} registros")
print(f" PAIS_BLOCO: {pais_bloco_tratado.count()} registros")

In [0]:
# Realizar join para criar dimensão geografia
dim_geografia = (
    pais_tratado
    .join(pais_bloco_tratado, on="CO_PAIS", how="left")
    .fillna("Não Definido", subset=["NO_BLOCO", "CO_BLOCO"])
    .withColumn("sk_geografia", sha2(concat_ws("||", col("CO_PAIS")), 256))
    .withColumn("data_criacao", current_timestamp())
)

print(f" Dimensão GEOGRAFIA criada: {dim_geografia.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    dim_geografia
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_gold}dim_geografia/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_geografia
    USING DELTA
    LOCATION '{path_storage_gold}dim_geografia/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("dim_geografia", dim_geografia.count())

print(" Dimensão GEOGRAFIA salva em Gold")

### Fato BALANCA_COMERCIAL_EXP

In [0]:
print("=" * 60)
print("CRIANDO FATO BALANCA_COMERCIAL_EXP")
print("=" * 60)

ultima_transformacao_exp = obter_data_ultima_transformacao_gold("fato_balanca_exp")

# Ler dados de EXP da Silver
df_exp = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/exp_tratado")

if ultima_transformacao_exp:
    df_exp = df_exp.filter(col("DATA_TRANSFORMACAO") > ultima_transformacao_exp)
    print(f" Filtrando EXP desde: {ultima_transformacao_exp}")
else:
    print(" Primeira execução: processando todos os EXP")

print(f"Total de registros EXP para processar: {df_exp.count()}")

In [0]:
# Criar fato com joins às dimensões
fato_exp = (
    df_exp
    .join(dim_tempo, on=["CO_ANO", "CO_MES"], how="left")
    .join(dim_produto, on="CO_NCM", how="left")
    .join(dim_geografia, on="CO_PAIS", how="left")
    .select(
        col("sk_tempo").alias("fk_tempo"),
        col("sk_produto").alias("fk_produto"),
        col("sk_geografia").alias("fk_geografia"),
        col("CO_ANO"),
        col("CO_MES"),
        col("CO_NCM"),
        col("CO_PAIS"),
        col("SG_UF_NCM"),
        col("QT_ESTAT").alias("quantidade_estatistica"),
        col("KG_LIQUIDO").alias("peso_liquido_kg"),
        col("VL_FOB").alias("valor_fob"),
        col("TIPO_OPERACAO"),
        current_timestamp().alias("data_fato")
    )
)

print(f" Fato EXP criado: {fato_exp.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    fato_exp
    .write
    .format("delta")
    .mode("append")
    .partitionBy("CO_ANO", "CO_MES")
    .save(f"{path_storage_gold}fato_balanca_exp/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.fato_balanca_exp
    USING DELTA
    LOCATION '{path_storage_gold}fato_balanca_exp/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("fato_balanca_exp", fato_exp.count())

print(" Fato BALANCA_COMERCIAL_EXP salvo em Gold")

### Fato BALANCA_COMERCIAL_IMP

In [0]:
print("=" * 60)
print("CRIANDO FATO BALANCA_COMERCIAL_IMP")
print("=" * 60)

ultima_transformacao_imp = obter_data_ultima_transformacao_gold("fato_balanca_imp")

# Ler dados de IMP da Silver
df_imp = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/imp_tratado")

if ultima_transformacao_imp:
    df_imp = df_imp.filter(col("DATA_TRANSFORMACAO") > ultima_transformacao_imp)
    print(f" Filtrando IMP desde: {ultima_transformacao_imp}")
else:
    print(" Primeira execução: processando todos os IMP")

print(f"Total de registros IMP para processar: {df_imp.count()}")

In [0]:
# Criar fato com joins às dimensões
fato_imp = (
    df_imp
    .join(dim_tempo, on=["CO_ANO", "CO_MES"], how="left")
    .join(dim_produto, on="CO_NCM", how="left")
    .join(dim_geografia, on="CO_PAIS", how="left")
    .select(
        col("sk_tempo").alias("fk_tempo"),
        col("sk_produto").alias("fk_produto"),
        col("sk_geografia").alias("fk_geografia"),
        col("CO_ANO"),
        col("CO_MES"),
        col("CO_NCM"),
        col("CO_PAIS"),
        col("SG_UF_NCM"),
        col("QT_ESTAT").alias("quantidade_estatistica"),
        col("KG_LIQUIDO").alias("peso_liquido_kg"),
        col("VL_FOB").alias("valor_fob"),
        col("VL_FRETE").alias("valor_frete"),
        col("VL_SEGURO").alias("valor_seguro"),
        col("TIPO_OPERACAO"),
        current_timestamp().alias("data_fato")
    )
)

print(f" Fato IMP criado: {fato_imp.count()} registros")

In [0]:
# Escrita incremental (append) na Gold
(
    fato_imp
    .write
    .format("delta")
    .mode("append")
    .partitionBy("CO_ANO", "CO_MES")
    .save(f"{path_storage_gold}fato_balanca_imp/")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.fato_balanca_imp
    USING DELTA
    LOCATION '{path_storage_gold}fato_balanca_imp/'
""".format(path_storage_gold=path_storage_gold))

registrar_transformacao_gold("fato_balanca_imp", fato_imp.count())

print(" Fato BALANCA_COMERCIAL_IMP salvo em Gold")

### Validações e Estatísticas

In [0]:
print("=" * 60)
print("VALIDAÇÕES GOLD")
print("=" * 60)

# Validação dimensões
print("\n📊 DIMENSÕES:")
print(f" DIM_TEMPO: {spark.table('gold.dim_tempo').count()} registros")
print(f" DIM_PRODUTO: {spark.table('gold.dim_produto').count()} registros")
print(f" DIM_LOCALIDADE_BR: {spark.table('gold.dim_localidade_br').count()} registros")
print(f" DIM_GEOGRAFIA: {spark.table('gold.dim_geografia').count()} registros")

# Validação fatos
print("\n📊 FATOS:")
print(f" FATO_BALANCA_EXP: {spark.table('gold.fato_balanca_exp').count()} registros")
print(f" FATO_BALANCA_IMP: {spark.table('gold.fato_balanca_imp').count()} registros")

In [0]:
# Amostra de dados
print("\n AMOSTRA DIM_TEMPO:")
spark.table("gold.dim_tempo").limit(5).display()

In [0]:
print("\n AMOSTRA DIM_PRODUTO:")
spark.table("gold.dim_produto").limit(5).display()

In [0]:
print("\n AMOSTRA FATO_BALANCA_EXP:")
spark.table("gold.fato_balanca_exp").limit(5).display()

In [0]:
print("\n AMOSTRA FATO_BALANCA_IMP:")
spark.table("gold.fato_balanca_imp").limit(5).display()

### Estatísticas Finais

In [0]:
print("=" * 60)
print("ESTATÍSTICAS EXPORTAÇÃO (EXP)")
print("=" * 60)

df_stats_exp = spark.sql("""
    SELECT
        CO_ANO,
        CO_MES,
        COUNT(*) as total_registros,
        SUM(quantidade_estatistica) as total_quantidade,
        SUM(peso_liquido_kg) as total_peso_kg,
        SUM(valor_fob) as total_valor_fob,
        MAX(data_fato) as ultima_atualizacao
    FROM gold.fato_balanca_exp
    GROUP BY CO_ANO, CO_MES
    ORDER BY CO_ANO DESC, CO_MES DESC
""")

df_stats_exp.display()

In [0]:
print("=" * 60)
print("ESTATÍSTICAS IMPORTAÇÃO (IMP)")
print("=" * 60)

df_stats_imp = spark.sql("""
    SELECT
        CO_ANO,
        CO_MES,
        COUNT(*) as total_registros,
        SUM(quantidade_estatistica) as total_quantidade,
        SUM(peso_liquido_kg) as total_peso_kg,
        SUM(valor_fob) as total_valor_fob,
        SUM(valor_frete) as total_frete,
        SUM(valor_seguro) as total_seguro,
        MAX(data_fato) as ultima_atualizacao
    FROM gold.fato_balanca_imp
    GROUP BY CO_ANO, CO_MES
    ORDER BY CO_ANO DESC, CO_MES DESC
""")

df_stats_imp.display()

In [0]:
print("=" * 60)
print("RESUMO DAS TRANSFORMAÇÕES GOLD")
print("=" * 60)

df_resumo_gold = spark.sql("""
    SELECT
        tabela_gold,
        COUNT(*) as total_transformacoes,
        SUM(total_registros) as total_registros_processados,
        MAX(data_ultima_transformacao) as ultima_transformacao
    FROM gold.transformacoes_processadas
    GROUP BY tabela_gold
    ORDER BY tabela_gold
""")

df_resumo_gold.display()

In [0]:
print("\n Transformações Gold finalizadas com sucesso!")